# A Demo and Comparison of Categorical Methods

**The feature engineering step is arguably the most intensive and manual component of designing and completing a machine learning model from a dataset. Within feature engineering itself, categorical encoding is an essential part of nearly every dataset, but often comes with its own challenges.**

In this notebook, we will be exploring where and when to apply different categorical methods. Specifically, we will be applying a number of categorical methods to a dataset, and then comparing the resulting performance of each of the categorical methods. For a detailed guide on the when/how of using categorical methods, check out [this notebook](

Specifically for this demo, we will be using the dataset located at https://www.kaggle.com/c/instacart-market-basket-analysis/data. For the purposes of this demo, we will not be detailing every single step involved in the machine learning process. For that, check out [this comprehensive demo](https://github.com/Featuretools/predict-next-purchase) by Feature Labs, Inc. We will still be outlining several key steps, however.

If you are planning to follow this tutorial yourself, please refer to [this link](https://github.com/Featuretools/predict-next-purchase#running-the-tutorial) for instructions to download/prepare the data. 